# Axial Annular Flow - Power Law

![caption](frederickson_bird_ax_ann_flow_fig.png?raw=1)

![caption](hanks_larsen_ax_ann_flow.png?raw=1)

In [26]:
from numpy import *
from scipy.integrate import quad
from scipy.optimize import fsolve

In [23]:

class beta():
    def betaeqn(self,beta):
        return self.intlhs(beta) - self.intrhs(beta)
    def lhs(self, beta, xi):
        return ( beta**2/xi - xi)**(1.0/self.n)
    def rhs(self,beta,xi):
        return (xi-beta**2/xi)**(1.0/self.n)
    def intlhs(self, beta):
        return quad(lambda xi: self.lhs(beta,xi),self.kappa, beta)[0]
    def intrhs(self, beta):
        return quad(lambda xi: self.rhs(beta,xi),beta, 1.0)[0]
    def qeqn(self, dplguess):
        self.dpl = dplguess
        beta = self.betacomp()
        return self.qdef - 3.14159265359*self.rad**3/(1.0/self.n+3.0)* \
                    (self.rad/(2.0*self.k)*dplguess)**(1.0/self.n)*((1.0-beta**2)** \
                    (1.0+1.0/self.n)-self.kappa**(1.0-1.0/self.n)*(beta**2-self.kappa**2) \
                    **(1.0+1.0/self.n))
    
class anncomp(beta):
    def betacomp(self):
        self.betaguess = self.kappa + (1.0-self.kappa)/2.0
        self.beta = fsolve( self.betaeqn, self.betaguess)
        return self.beta
    def dplcomp(self):
        #need set self.dplguess to solve for dpl given q; q is specified as self.qdef
        self.dpl = fsolve( self.qeqn, self.dplguess)
        self.betacomp()
        return self.dpl  
    def q(self):
        return 3.14159265359*self.rad**3/(1.0/self.n+3.0)*(self.rad/(2.0*self.k)*self.dpl)** \
                (1.0/self.n)*((1.0-self.beta**2)**(1.0+1.0/self.n)-self.kappa**(1.0-1.0/self.n)* \
                (self.beta**2-self.kappa**2)**(1.0+1.0/self.n))
    def stress(self):
        self.stressinner = self.dpl*self.rad/2.0*(self.kappa-self.beta**2/self.kappa)
        self.stressouter = self.dpl*self.rad/2.0*(1.0-self.beta**2)
        return (self.stressinner, self.stressouter)
    def shearrate(self):
        self.rateinner = (-self.stressinner/self.k)**(1.0/self.n)
        self.rateouter = (self.stressouter/self.k)**(1.0/self.n)
        return ( self.rateinner, self.rateouter )
        

class anntest(anncomp):
    #def __init__(self, kappa=0.974, k=892.0, n=0.1155, dpl=1.5e+7, rad=0.02):
    #    self.kappa=kappa; self.k=k; self.n=n; self.dpl=dpl; self.rad=rad;
    def getdpl(self):
        # this method runs through solving for q given dpl
        self.dplcomp()
        print('beta = ', self.betacomp())
        print('kappa = ', self.kappa)
        print('n = ', self.n)
        print('Pressure drop per unit length (solved) = ',self.dpl)
        print('Computed volumetric flow rate = ',self.q())
        stress = self.stress()
        print('Wall shear rates = ', self.shearrate())
        return [ self.betacomp(), self.kappa, self.n ]
    def getq(self):
        # this method computes q given dpl
        self.betacomp()
        print('beta = ', self.beta)
        print('kappa = ', self.kappa)
        print('n = ', self.n)
        print('Pressure drop per unit length (given) = ',self.dpl)
        print('Computed volumetric flow rate = ',self.q())
        stress = self.stress()
        print('Wall shear rates = ', self.shearrate())
        return [ self.betacomp(), self.kappa, self.n ]
    def postresult(self):
        return [ self.q(), self.dpl, self.stress(), self.shearrate() ]


In [25]:

# Instantiate object
a = anntest()

# Define variables
# outer radius
a.rad = 15.0e-3
# kappa
a.kappa = 9.75e-3/a.rad
# power law k #inner divided by outer radius
a.k = 250.0
# power law n
a.n = 0.4

# Use this to solve for dp/L given q
#flow rate in cubic meters per sec
a.qdef = 5.77e-4
a.dplguess = 5.0 / 14.7 *101325.0 / 3.0e-3
a.getdpl()

# Use this to compute q given dp/L
#a.dpl = 14819641.7282
#a.getq()

beta =  [0.81452102]
kappa =  0.65
n =  0.4
Pressure drop per unit length (solved) =  [2144701.19280107]
Computed volumetric flow rate =  [0.000577]
Wall shear rates =  (array([2777.98099986]), array([2182.03569587]))


[array([0.81452102]), 0.65, 0.4]